In [2]:
import mysql.connector

mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password="jujo",
  auth_plugin='mysql_native_password'
)

mycursor = mydb.cursor()

In [3]:
mycursor.execute("use crypto_analysis")

In [5]:
query = """
    SELECT  
        AVG(h.price_usd) AS btc_avg, 
        AVG(o.price) AS oil_avg
    FROM historical_Prices h
    LEFT JOIN oil_prices o ON h.date = o.date
    WHERE h.coin_id = 'bitcoin' 
      AND h.date >= '2025-01-01' 
      AND h.date <= '2025-12-31';
"""

mycursor.execute(query)
result = mycursor.fetchone() 

print("--- 2025 Avg Analysis ---")

if result and result[0] is not None:
    btc_avg = result[0]
    oil_avg = result[1] if result[1] is not None else 0
    print(f"  BTC Avg: ${btc_avg:,.2f}")
    print(f"  Oil Avg: ${oil_avg:,.2f}")
else:
    print("  No data found for the year 2025.")
    print("  Hint: Check if your table has dates like '2025-01-01'")

--- 2025 Avg Analysis ---
  BTC Avg: $101,952.83
  Oil Avg: $64.23


In [ ]:
import date 

In [4]:
query_2= """
    SELECT  
    CAST(btc.date AS DATE) AS "Date",
        btc.price_usd AS btc_price,
        sp.close AS sp500_close
    FROM historical_Prices btc
    INNER JOIN stock_prices sp ON btc.date = sp.date
    WHERE btc.coin_id = 'bitcoin' 
      AND sp.ticker = 'GSPC'
      AND btc.date >= '2024-01-01'
    ORDER BY btc.date ASC
"""

mycursor.execute(query_2)
result = mycursor.fetchone() 

for row in result:
        
        print(f"Date: {row[0]} | btc_price {row[1]:,.2f} | sp500_close: {row[2]:,.2f}")

TypeError: 'datetime.date' object is not subscriptable

In [38]:
query_3= """SELECT 
    c.date, 
    c.price_usd AS ethereum_price, 
    s.close AS nasdaq_close
FROM historical_Prices c
JOIN stock_prices s ON c.date = s.date
WHERE c.coin_id = 'ethereum' 
  AND s.ticker = 'IXIC' 
  AND YEAR(c.date) = 2025
ORDER BY c.date;
"""
mycursor.execute(query_3)
result = mycursor.fetchall()

for row in result:
        
        print(f"Date: {row[0]} | Ethereum: {row[1]:,.2f} | NASDAQ: {row[2]:,.2f}")

Date: 2025-02-12 | Ethereum: 2,603.03 | NASDAQ: 19,649.95
Date: 2025-02-13 | Ethereum: 2,736.27 | NASDAQ: 19,945.64
Date: 2025-02-14 | Ethereum: 2,675.71 | NASDAQ: 20,026.77
Date: 2025-02-18 | Ethereum: 2,741.91 | NASDAQ: 20,041.26
Date: 2025-02-19 | Ethereum: 2,669.49 | NASDAQ: 20,056.25
Date: 2025-02-20 | Ethereum: 2,714.80 | NASDAQ: 19,962.36
Date: 2025-02-21 | Ethereum: 2,741.59 | NASDAQ: 19,524.01
Date: 2025-02-24 | Ethereum: 2,827.18 | NASDAQ: 19,286.93
Date: 2025-02-25 | Ethereum: 2,503.10 | NASDAQ: 19,026.39
Date: 2025-02-27 | Ethereum: 2,325.85 | NASDAQ: 18,544.42
Date: 2025-02-28 | Ethereum: 2,305.32 | NASDAQ: 18,847.28
Date: 2025-03-03 | Ethereum: 2,517.34 | NASDAQ: 18,350.19
Date: 2025-03-04 | Ethereum: 2,148.20 | NASDAQ: 18,285.16
Date: 2025-03-05 | Ethereum: 2,171.50 | NASDAQ: 18,552.73
Date: 2025-03-06 | Ethereum: 2,241.38 | NASDAQ: 18,069.26
Date: 2025-03-07 | Ethereum: 2,202.32 | NASDAQ: 18,196.22
Date: 2025-03-10 | Ethereum: 2,017.27 | NASDAQ: 17,468.32
Date: 2025-03-

In [15]:
# Wrap the SQL in triple quotes so Python treats it as a string
query_4 = """
WITH OilChanges AS (
    SELECT 
        `date`, 
        Price,
        LAG(Price) OVER (ORDER BY `date`) AS prev_oil_price,
        ((Price - LAG(Price) OVER (ORDER BY `date`)) / LAG(Price) OVER (ORDER BY `date`)) * 100 AS oil_pct_change
    FROM oil_prices
),
BTCChanges AS (
    SELECT 
        `date`, 
        price_usd,
        ((price_usd - LAG(price_usd) OVER (ORDER BY `date`)) / LAG(price_usd) OVER (ORDER BY `date`)) * 100 AS btc_pct_change
    FROM historical_Prices
    WHERE coin_id = 'bitcoin'
)
SELECT 
    o.`date`,
    o.oil_pct_change AS oil_spike_pct,
    b.btc_pct_change AS btc_response_pct
FROM OilChanges o
JOIN BTCChanges b ON o.`date` = b.`date`
WHERE o.oil_pct_change > 5  
ORDER BY o.oil_pct_change DESC;
"""

# Now execute it
mycursor.execute(query_4)
result = mycursor.fetchall()

for row in result:
        print(f"Date: {row[0]} | price_usd: ${row[1]:,.2f} ")

Date: 2025-06-13 | price_usd: $7.43 
Date: 2025-10-23 | price_usd: $5.30 


In [ ]:
#Query 5 Compare top 3 coins daily price trend vs Nifty (^NSEI).

In [23]:
query_5 = """SELECT 
    h.date, 
    h.coin_id, 
    h.price_usd AS crypto_price, 
    o.Price AS oil_price, 
    s.Close AS nifty_price
FROM historical_Prices h
LEFT JOIN oil_prices o ON h.date = o.date
LEFT JOIN stock_prices s ON h.date = s.date AND s.Ticker = 'NSEI'
WHERE h.coin_id IN ('bitcoin', 'ethereum', 'tether')
ORDER BY h.date ASC;
"""
mycursor.execute(query_5)
result = mycursor.fetchall()

for row in result:
        print(f"Date: {row[0]} | coin_id: ${row[1]:} | price_usd: {row[2]:,.2f} ")

Date: 2025-02-12 | coin_id: $bitcoin | price_usd: 95,739.98 
Date: 2025-02-12 | coin_id: $ethereum | price_usd: 2,603.03 
Date: 2025-02-12 | coin_id: $tether | price_usd: 1.00 
Date: 2025-02-13 | coin_id: $bitcoin | price_usd: 97,836.19 
Date: 2025-02-13 | coin_id: $ethereum | price_usd: 2,736.27 
Date: 2025-02-13 | coin_id: $tether | price_usd: 1.00 
Date: 2025-02-14 | coin_id: $bitcoin | price_usd: 96,561.66 
Date: 2025-02-14 | coin_id: $ethereum | price_usd: 2,675.71 
Date: 2025-02-14 | coin_id: $tether | price_usd: 1.00 
Date: 2025-02-15 | coin_id: $bitcoin | price_usd: 97,488.48 
Date: 2025-02-15 | coin_id: $ethereum | price_usd: 2,724.67 
Date: 2025-02-15 | coin_id: $tether | price_usd: 1.00 
Date: 2025-02-16 | coin_id: $bitcoin | price_usd: 97,569.95 
Date: 2025-02-16 | coin_id: $ethereum | price_usd: 2,692.82 
Date: 2025-02-16 | coin_id: $tether | price_usd: 1.00 
Date: 2025-02-17 | coin_id: $bitcoin | price_usd: 96,149.35 
Date: 2025-02-17 | coin_id: $ethereum | price_usd: 2,6

In [22]:
query_6 = """SELECT 
    s.`date`,
    s.Close AS sp500_close,
    o.Price AS oil_price,
    -- Calculate Daily % Change for S&P 500
    ((s.Close - LAG(s.Close) OVER (ORDER BY s.`date`)) / LAG(s.Close) OVER (ORDER BY s.`date`)) * 100 AS sp500_pct_change,
    -- Calculate Daily % Change for Oil
    ((o.Price - LAG(o.Price) OVER (ORDER BY o.`date`)) / LAG(o.Price) OVER (ORDER BY o.`date`)) * 100 AS oil_pct_change
FROM stock_prices s
JOIN oil_prices o ON s.`date` = o.`date`
WHERE s.Ticker = 'GSPC'
ORDER BY s.`date` ASC;
"""
mycursor.execute(query_6)
result = mycursor.fetchall()

for row in result:
        print(f"Date: {row[0]} | Close: ${row[1]:} | Price: {row[2]:,.2f} ")

Date: 2020-01-02 | Close: $3257.850098 | Price: 61.17 
Date: 2020-01-03 | Close: $3234.850098 | Price: 63.00 
Date: 2020-01-06 | Close: $3246.280029 | Price: 63.27 
Date: 2020-01-07 | Close: $3237.179932 | Price: 62.70 
Date: 2020-01-08 | Close: $3253.050049 | Price: 59.65 
Date: 2020-01-09 | Close: $3274.699951 | Price: 59.56 
Date: 2020-01-10 | Close: $3265.350098 | Price: 59.02 
Date: 2020-01-13 | Close: $3288.129883 | Price: 58.17 
Date: 2020-01-14 | Close: $3283.149902 | Price: 58.34 
Date: 2020-01-15 | Close: $3289.290039 | Price: 57.86 
Date: 2020-01-16 | Close: $3316.810059 | Price: 58.52 
Date: 2020-01-17 | Close: $3329.620117 | Price: 58.55 
Date: 2020-01-21 | Close: $3320.790039 | Price: 58.25 
Date: 2020-01-22 | Close: $3321.750000 | Price: 56.76 
Date: 2020-01-23 | Close: $3325.540039 | Price: 55.51 
Date: 2020-01-24 | Close: $3295.469971 | Price: 54.09 
Date: 2020-01-27 | Close: $3243.629883 | Price: 53.09 
Date: 2020-01-28 | Close: $3276.239990 | Price: 53.33 
Date: 2020

In [24]:
query_7 = """SELECT 
    h.`date`,
    h.price_usd AS btc_price,
    o.Price AS oil_price
FROM historical_Prices h
JOIN oil_prices o ON h.`date` = o.`date`
WHERE h.coin_id = 'bitcoin'
ORDER BY h.`date` ASC;
"""
mycursor.execute(query_7)
result = mycursor.fetchall()

for row in result:
        print(f"Date: {row[0]} | btc_price: ${row[1]:} | oil_price: {row[2]:,.2f} ")

Date: 2025-02-12 | btc_price: $95739.977371 | oil_price: 71.72 
Date: 2025-02-13 | btc_price: $97836.188561 | oil_price: 71.66 
Date: 2025-02-14 | btc_price: $96561.663999 | oil_price: 71.05 
Date: 2025-02-18 | btc_price: $95776.157239 | oil_price: 72.21 
Date: 2025-02-19 | btc_price: $95495.891533 | oil_price: 72.58 
Date: 2025-02-20 | btc_price: $96554.871563 | oil_price: 72.88 
Date: 2025-02-21 | btc_price: $98384.317382 | oil_price: 70.72 
Date: 2025-02-24 | btc_price: $96327.463091 | oil_price: 71.06 
Date: 2025-02-25 | btc_price: $91396.766869 | oil_price: 69.15 
Date: 2025-02-26 | btc_price: $88755.769336 | oil_price: 68.87 
Date: 2025-02-27 | btc_price: $83900.114965 | oil_price: 70.62 
Date: 2025-02-28 | btc_price: $84709.144778 | oil_price: 69.97 
Date: 2025-03-03 | btc_price: $94261.532865 | oil_price: 68.63 
Date: 2025-03-04 | btc_price: $86124.714187 | oil_price: 68.47 
Date: 2025-03-05 | btc_price: $87310.805311 | oil_price: 66.58 
Date: 2025-03-06 | btc_price: $90604.080

In [28]:
query_8 = """SELECT 
    s.`date`,
    s.Close AS nasdaq_close,
    h.price_usd AS eth_price,
    -- Daily % Change for NASDAQ
    ((s.Close - LAG(s.Close) OVER (ORDER BY s.`date`)) / LAG(s.Close) OVER (ORDER BY s.`date`)) * 100 AS nasdaq_pct_change,
    -- Daily % Change for Ethereum
    ((h.price_usd - LAG(h.price_usd) OVER (ORDER BY h.`date`)) / LAG(h.price_usd) OVER (ORDER BY h.`date`)) * 100 AS eth_pct_change
FROM stock_prices s
JOIN historical_Prices h ON s.`date` = h.`date`
WHERE s.Ticker = 'IXIC' AND h.coin_id = 'ethereum'
ORDER BY s.`date` ASC;
"""
mycursor.execute(query_8)
result = mycursor.fetchall()

for row in result:
        print(f"Date: {row[0]} | nasdaq_close: ${row[1]} | eth_price: {row[2]:,.2f} ")

Date: 2025-02-12 | nasdaq_close: $19649.949219 | eth_price: 2,603.03 
Date: 2025-02-13 | nasdaq_close: $19945.640625 | eth_price: 2,736.27 
Date: 2025-02-14 | nasdaq_close: $20026.769531 | eth_price: 2,675.71 
Date: 2025-02-18 | nasdaq_close: $20041.259766 | eth_price: 2,741.91 
Date: 2025-02-19 | nasdaq_close: $20056.250000 | eth_price: 2,669.49 
Date: 2025-02-20 | nasdaq_close: $19962.359375 | eth_price: 2,714.80 
Date: 2025-02-21 | nasdaq_close: $19524.009766 | eth_price: 2,741.59 
Date: 2025-02-24 | nasdaq_close: $19286.929688 | eth_price: 2,827.18 
Date: 2025-02-25 | nasdaq_close: $19026.390625 | eth_price: 2,503.10 
Date: 2025-02-27 | nasdaq_close: $18544.419922 | eth_price: 2,325.85 
Date: 2025-02-28 | nasdaq_close: $18847.279297 | eth_price: 2,305.32 
Date: 2025-03-03 | nasdaq_close: $18350.189453 | eth_price: 2,517.34 
Date: 2025-03-04 | nasdaq_close: $18285.160156 | eth_price: 2,148.20 
Date: 2025-03-05 | nasdaq_close: $18552.730469 | eth_price: 2,171.50 
Date: 2025-03-06 | n

In [31]:
query_9 = """SELECT 
    h.`date`,
    h.coin_id,
    h.price_usd AS crypto_price,
    s_nasdaq.Close AS nasdaq_close,
    s_sp500.Close AS sp500_close
FROM historical_Prices h
-- Join NASDAQ (IXIC)
LEFT JOIN stock_prices s_nasdaq 
    ON h.`date` = s_nasdaq.`date` AND s_nasdaq.Ticker = 'IXIC'
-- Join S&P 500 (GSPC)
LEFT JOIN stock_prices s_sp500 
    ON h.`date` = s_sp500.`date` AND s_sp500.Ticker = 'GSPC'
WHERE h.coin_id IN ('bitcoin', 'ethereum', 'tether')
  AND h.`date` BETWEEN '2025-01-01' AND '2025-12-31'
ORDER BY h.`date` ASC, h.coin_id;
"""
mycursor.execute(query_9)
result = mycursor.fetchall()

for row in result:
        print(f"Date: {row[0]} | coin_id: ${row[1]} | crypto_price: {row[2]} | nasdaq_close: {row[3]} | sp500_close: {row[2]:,.2f} ")

Date: 2025-02-12 | coin_id: $bitcoin | crypto_price: 95739.977371 | nasdaq_close: 19649.949219 | sp500_close: 95,739.98 
Date: 2025-02-12 | coin_id: $ethereum | crypto_price: 2603.033370 | nasdaq_close: 19649.949219 | sp500_close: 2,603.03 
Date: 2025-02-12 | coin_id: $tether | crypto_price: 0.999842 | nasdaq_close: 19649.949219 | sp500_close: 1.00 
Date: 2025-02-13 | coin_id: $bitcoin | crypto_price: 97836.188561 | nasdaq_close: 19945.640625 | sp500_close: 97,836.19 
Date: 2025-02-13 | coin_id: $ethereum | crypto_price: 2736.270314 | nasdaq_close: 19945.640625 | sp500_close: 2,736.27 
Date: 2025-02-13 | coin_id: $tether | crypto_price: 1.000027 | nasdaq_close: 19945.640625 | sp500_close: 1.00 
Date: 2025-02-14 | coin_id: $bitcoin | crypto_price: 96561.663999 | nasdaq_close: 20026.769531 | sp500_close: 96,561.66 
Date: 2025-02-14 | coin_id: $ethereum | crypto_price: 2675.709001 | nasdaq_close: 20026.769531 | sp500_close: 2,675.71 
Date: 2025-02-14 | coin_id: $tether | crypto_price: 1.0

In [37]:
query_10 = """SELECT 
    h.`date`,
    h.price_usd AS btc_price,
    s.Close AS sp500_close,
    o.Price AS oil_price,
    -- Calculate Daily % Change for BTC to see correlation in volatility
    ((h.price_usd - LAG(h.price_usd) OVER (PARTITION BY h.coin_id ORDER BY h.`date`)) 
        / LAG(h.price_usd) OVER (PARTITION BY h.coin_id ORDER BY h.`date`)) * 100 AS btc_daily_return
FROM historical_Prices h
-- Join S&P 500
LEFT JOIN stock_prices s 
    ON h.`date` = s.`date` AND s.Ticker = '^GSPC'
-- Join Crude Oil
LEFT JOIN oil_prices o 
    ON h.`date` = o.`date`
WHERE h.coin_id = 'bitcoin'
  AND h.`date` BETWEEN '2025-01-01' AND '2025-12-31'
ORDER BY h.`date` ASC;
"""
mycursor.execute(query_10)
result = mycursor.fetchall()

for row in result:
        print(f"Date: {row[0]} | btc_price: ${row[1]} | sp500_close: {row[2]} | oil_price: {row[3]:,.2f} ")

Date: 2025-02-12 | btc_price: $95739.977371 | sp500_close: None | oil_price: 71.72 
Date: 2025-02-13 | btc_price: $97836.188561 | sp500_close: None | oil_price: 71.66 
Date: 2025-02-14 | btc_price: $96561.663999 | sp500_close: None | oil_price: 71.05 


TypeError: unsupported format string passed to NoneType.__format__